
# Training ClassicVC

Lyodos 著

Version 1.0.0 (2024-07-14)

前提は、前のノートブックを実行してデータセットの音声をダウンロード＆フォーマット変換し、メタデータを作成済みであること。

まず、チェックポイントやログ等を書き出すためのフォルダを設定する。




In [ ]:
%%time
from pathlib import Path

# チェックポイントやログを保存する、プロジェクト単位および訓練ジョブ単位のルートディレクトリの指定

DATASET_ROOT_PATH = Path("/home/lyodos/study/dataset") # このフォルダ名はユーザーの実情に合わせて書き変えること

# まず、プロジェクト（ClassicVC）全体で保存先を決める。ちなみに使うデータセットの組み合わせでサブディレクトリを作っている。
proj_path = DATASET_ROOT_PATH / "checkpoints" / "classic-vc" / "voxceleb12_libri_samr"
proj_path.mkdir(parents = True, exist_ok = True)
print("Project directory:", str(proj_path))

# 訓練ジョブに名前を付ける
JOB_NAME = "VC01"

# チェックポイントとログの保存先を定義する。proj_path の下にジョブ名でフォルダを作る。
ckpt_path = Path(proj_path) / JOB_NAME
ckpt_path.mkdir(parents = True, exist_ok = True)
print("Training job directory:", str(ckpt_path))

logs_path = ckpt_path / "tensorboard" # TensorBoard 用のログは数が多くなるので、ジョブ名の下にサブフォルダを作ってまとめる。
logs_path.mkdir(parents = True, exist_ok = True)



次に logger （TensorBoard ではなく Notebook 内の情報を確認するためのもの）を作成する。


In [ ]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) #出力レベルの設定

for h in logger.handlers[:]:
    logger.removeHandler(h) # logger を新しく作成した場合、既存の handler があれば全て除去する。

file_handler = logging.FileHandler(ckpt_path / 'classic-vc.log') # ログファイル用ハンドラの生成
sout_handler = logging.StreamHandler() # 標準出力用ハンドラの生成

fmt = logging.Formatter('%(asctime)s %(message)s') # フォーマッタの生成
file_handler.setFormatter(fmt) # ハンドラにフォーマッタを登録
sout_handler.setFormatter(fmt) # ハンドラにフォーマッタを登録

if not logger.hasHandlers():
    logger.addHandler(file_handler) # ロガーにハンドラを登録
    logger.addHandler(sout_handler)

logger.info('Checkpoints are saved to: {}'.format(str(ckpt_path)))
logger.info('TensorBoard log is saved to : {}'.format(str(logs_path)))



訓練に使う GPU を指定する。現在マルチ GPU には対応していない。


In [ ]:
import torch

if torch.cuda.device_count() >= 2:
    device = torch.device('cuda:0')
elif torch.cuda.device_count() >= 1:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

logger.info('PyTorch device: {}'.format(device))
logger.info('PyTorch device name: {}'.format(torch.cuda.get_device_name(device = device)))



以下、ネットワークの部品を定義していく。


----

## HarmoF0 pitch tracker の定義


* 入力は `torch.Size([n_batch, n_sample])` ただし `(1, 513)` （32.1 ms）以上の wave length がないとエラーになる。


In [ ]:
import torch
import torchaudio
import sys
sys.path.append('../') # ClassicVC のリポジトリのルートをパスに入れて、model ディレクトリを探せるようにしている

from model.harmof0.pitch_tracker import BatchedPitchEnergyTracker

def pred_f0_len(length):
    return length // 160 + 1

harmof0_tracker = BatchedPitchEnergyTracker(
    checkpoint_path = "../model/harmof0/checkpoints/mdb-stem-synth.pth", # HarmoF0 作者による訓練済みの重みを再配布
    fmin = 27.5, # f0 として想定する最低周波数の Hz で、ピアノの最低音の A に相当する。
    sample_rate = 16000,
    hop_length = 160, # f0 を推定する間隔。160/16000 = 10 ms 
    frame_len = 1024, # sliding window を切り出す長さ
    frames_per_step = 1000, # 1 回の forward で投入する最大セグメント数
    high_threshold = 0.8, 
    low_threshold = 0.1, 
    freq_bins_in = 88*4,
    bins_per_octave_in = 48,
    bins_per_octave_out = 48,
    device = device,
    compile = False,
    dry_run = 10, 
)

num_harmof0_params = sum([p.numel() for p in list(harmof0_tracker.single_tracker.net.parameters())])
logger.info(f"Pitch tracker has {num_harmof0_params} parameters.")


----

## (Acoustic) Style Encoder の初期化

$E_{a}(x)$ を作成する。
入力はスペクトログラムで、さらに `(batch, 1, dim_spec, n_frame >= 80)` の 4D テンソルでないと受けられない。

なお HarmoF0 の低音側は 27.5 Hz だが、下の 1 オクターブ（48 bins）は人の声で使うことはなく無駄なので、
入力は 352 ではなく 352 - 48 = 304 にする。これでパラメータ数を 23,719,264 → 23,189,104  に削減できる。

出力は時間次元を持たない `(batch, 128)`。


In [ ]:
from dataclasses import dataclass
import typing
from omegaconf import OmegaConf

from model.StyleTTS2.models import StyleEncoder


@dataclass
class StyleEncoderConfig:
    dim_in: int = 304
    style_dim: int = 128
    max_conv_dim: int = 512

style_encoder_cfg = OmegaConf.structured(StyleEncoderConfig())
with open(ckpt_path / "style_encoder_cfg.yaml", 'w') as handle:
    OmegaConf.save(config = style_encoder_cfg, f = handle)

style_encoder = StyleEncoder(
    dim_in = style_encoder_cfg.dim_in, # 304
    style_dim = style_encoder_cfg.style_dim, # 128
    max_conv_dim = style_encoder_cfg.max_conv_dim, # 512
).to(device)


num_style_encoder_params = sum([p.numel() for p in list(style_encoder.parameters())])
logger.info(f"Pitch tracker has {num_style_encoder_params} parameters.")



----

## Content encoder (ContentVec)


Content の抽出に使う ContentVec のネットワーク構造は、transformers パッケージの HubertModel を使う。

> 重みは ContentVec の公式で配布しているものを若干アレンジする必要があるので、Notebook 04 に書いてある手順を参照して用意すること。


* 入力は `torch.Size([n_batch, n_sample])` の 16 khz mono waveform だが、 `(1, 400)` （25 ms）以上ないとエラーになる。

* HubertModel の生出力は feature last であるが、ほとんどの下流工程は time last になるので注意。

* 出力サイズは `torch.Size([batch, ((length - 80) // 320), 768])` で定義される。

入力が 16000 Hz で 1/320 に間引くので **ContentVec の出力テンソルの hop は 20 ms である**。

* HarmoF0 pitch tracker は 10 ms hop なので、ちょうど端数以外は 2 倍のテンソルサイズとなる。



In [ ]:
from transformers import HubertConfig, HubertModel

CE = HubertModel(HubertConfig())

# この位置に作った重みを置いておく
contentvec_path = DATASET_ROOT_PATH / "checkpoints" / "classic-vc" / "contentvec_500_hubert.pth"

CE_dict = torch.load(str(contentvec_path), map_location = torch.device('cpu'))
CE.load_state_dict(CE_dict, strict = True)
CE.eval().to(device)

num_CE_params = sum([p.numel() for p in list(CE.parameters())])
logger.info(f"ContentVec has {num_CE_params} parameters.")


----

## ProsodyPredictor の定義

ContentVec の抽出した特徴量と、Style Encoder で抽出した話者スタイル $s_a$ をもとに、
F0 と energy の時間変化を予測するネットワークである。


In [ ]:
from pathlib import Path
from dataclasses import dataclass
import typing
from omegaconf import OmegaConf

from model.StyleTTS2.models import F0NPredictorAll

@dataclass
class PrododyPredictorConfig:
    style_dim: int = 128
    hidden_dim: int = 768
    n_layer: int = 3
    dropout: float = 0.2

prosody_predictor_cfg = OmegaConf.structured(PrododyPredictorConfig())

with open(ckpt_path / "prosody_predictor_cfg.yaml", 'w') as handle:
    OmegaConf.save(config = prosody_predictor_cfg, f = handle)

# 新造した F0NPredictorAll は time last で入れる。返り値はそれぞれ channel 次元が潰れた 2D の time last だが、時間解像度が 2 倍
f0n_predictor = F0NPredictorAll(
    style_dim = prosody_predictor_cfg.style_dim,
    d_hid = prosody_predictor_cfg.hidden_dim,
    nlayers = prosody_predictor_cfg.n_layer,
    dropout = prosody_predictor_cfg.dropout,
).to(device)

num_f0n_predictor_params = sum([p.numel() for p in list(f0n_predictor.parameters())])
logger.info(f"Prosody predictor has {num_f0n_predictor_params} parameters.")


---


# VC Decoder の定義

StyleTTS 2 には iSTFTNet ベースのデコーダもあるが、スピードと音質を考えて、HiFi-GAN ベースだけをとりあえず考える。

```python
y_rec = decoder(
    en, # content 情報 torch.Size([1, 512, 445]) # 最終次元が長さ依存
    F0_real, # F0 torch.Size([1, 890]) # content 特徴量の長さの 2 倍
    real_norm, # Energy torch.Size([1, 890]) # content 特徴量の長さの 2 倍
    s, # acoustic style は固定サイズ torch.Size([1, 128])
)
```


In [ ]:
from pathlib import Path
from dataclasses import dataclass
import typing
from omegaconf import OmegaConf
import math

from model.StyleTTS2.hifigan import Decoder

upsample_rate_list = [10, 4, 3, 2]

@dataclass
class DecoderConfig:
    sampling_rate: int = 24000
    dim_in: int = 768
    style_dim: int = 128
    upsample_rate_list: list = tuple(upsample_rate_list)
    upsample_kernel_list: list = tuple([i*2 for i in upsample_rate_list])
    upsample_total: int = math.prod(upsample_rate_list)*2
    upsample_initial_channel: int = 512
    harmonic_num: int = 8

decoder_cfg = OmegaConf.structured(DecoderConfig())

with open(ckpt_path / "decoder_cfg.yaml", 'w') as handle:
    OmegaConf.save(config = decoder_cfg, f = handle)

decoder = Decoder(
    sampling_rate = decoder_cfg.sampling_rate,
    dim_in = decoder_cfg.dim_in,
    style_dim = decoder_cfg.style_dim,
    resblock_kernel_sizes = [3, 7, 11], # ここは大多数のモデルで同じ設定値を採用している
    resblock_dilation_sizes = [[1, 3, 5], [1, 3, 5], [1, 3, 5]], # ここは大多数のモデルで同じ設定値を採用している
    upsample_rates = decoder_cfg.upsample_rate_list,
    upsample_initial_channel = decoder_cfg.upsample_initial_channel,
    upsample_kernel_sizes = decoder_cfg.upsample_kernel_list,
    harmonic_num = decoder_cfg.harmonic_num,
).to(device)

num_dec_params = sum([p.numel() for p in list(decoder.parameters())])
logger.info(f"Decoder has {num_dec_params} parameters.")



実は注意が必要で、
`F0_real`, `real_norm` は運用時は偶数のフレーム長でなければならない。
さもないと content が // 2 で短くなってしまうため、割り切れない長さになり、decoder 内の AdainResBlk1d でエラーが出る。



----


# VC 訓練用のデータセット

Notebook 02 で作成して保存したメタデータを、proj_path に置く。



In [ ]:
%%time

import mgzip
import pickle

meta_name = str(proj_path / "classic-vc-meta.pkl.gz")

with mgzip.open(meta_name, "rb") as f:
    metadata = pickle.load(f) # 訓練ルートフォルダに配置済みの、話者＆発話データ一覧をロード


In [ ]:
from rich.pretty import pprint 

print(len(metadata))
print(len(metadata[0]))
pprint(metadata[0][0])



詳細な訓練設定を作る。
なお learning rate は中途半端な下がり方だが、これはレガシーなコードが残っているだけなので、
たぶんスケジューラ自体を削除して固定値で訓練しても上手く行く。


In [ ]:

from dataclasses import dataclass
import typing
from omegaconf import OmegaConf
import random

from torch.utils.data import DataLoader

from model.dataset import VCDataset, WavCollator


@dataclass
class VCTrainConfig:
    seed: int = 42
    val_speakers: int = 128 # 総データセットのうち何話者分を valid に振り分けるか
    batch_size: int = 2 # train データセットを、1 回につき何話者ずつ振り出すか
    val_batch_size: int = 8 # val データセットを、1 回につき何話者ずつ振り出すか
    n_utterances: int = 1 # train/val データセットを、1 回 1 話者につき何発話ずつ振り出すか。
    sec: float = 3.0 # 1st は 3 秒だった。サンプル秒数を一意に定める場合はここに秒数（例： 2.0）を指定。None なら max_sec に合わせる
    min_sec: float = 2.0 # 教師データの wav が最低限満たすべき有効な秒数
    max_sec: float = 5.0 # sec = None の場合に最長限度とするサンプル秒数 = valid dataset の場合
    sr: int = decoder_cfg.sampling_rate # 訓練中に取り扱う最高精度のサンプリング周波数

    n_workers: int = 4 # データローダーに使うプロセス立ち上げ数。多すぎると RAM が尽きる
    fp16: bool = False # 半精度による高速化 → やたらと nan が出まくるので使わないことにした（ただし bf16 なら行けるっぽい）
    board_every: int = 50 # Tensorboard への反映。単位 steps
    valid_every: int = 1 # validation set による評価。単位 epochs
    save_every: int = 5 # 保存間隔。単位 epochs

    start_lr: float = 1e-4 # Learning settings
    max_lr: float = 1e-4 # 3.5e-5 を超えたあたりから val loss が暴れる。
    end_lr: float = 1e-5 # end は 3e-7 で試していたが、もっと大きくてもいいだろう。
    n_warmup_epochs: int = 10
    t_initial: int = 3000 # スケジューラの 1 サイクルのエポック数
    n_epochs: int = 1000 # 最大エポック数
    TMA_epoch: int = 5 # 1st stage training において最初の n epochs は TMA を行わず spec loss だけを計算する。デフォルト 5
    cycle_decay: float = 1.0 # 再起動ごとに max_lr の値に倍数をかける。100万ステップ（5 回）で 1e-5 まで落としたい
    k_decay: float = 1.0 # CosineLR の低下速度。1.0 が標準で、 0 < k_decay < 1 だと速く落ちる。
    cycle_limit: int = 10 # 上限エポックまでに、何回のサイクルを回すか。1 なら再起動なし
    betas: typing.Tuple[float, float] = (0.8, 0.99) # HiFi-GAN に合わせた
    grad_clip: float = 10.0
    f0_act_threshold: float = 0.7 # HarmoF0 の activation に基づき、有声部のみを損失計算に使う
    lambda_spec: float = 5.0 # スペクトログラム再構成損失の係数
    lambda_F0: float = 0.1 # F0 の再構成損失の係数
    lambda_norm: float = 1.0 # Energy の再構成損失の係数
    lambda_gen: float = 1.0 # loss_gen_all の係数 = generator loss
    lambda_slm: float = 1.0 # loss_slm の係数 = slm feature matching loss
    lambda_sty: float = 1.0 # loss_sty の係数

tr_cfg = OmegaConf.structured(VCTrainConfig())

with open(ckpt_path / "train_cfg.yaml", 'w') as handle:
    OmegaConf.save(config = tr_cfg, f = handle)


# 辞書形式の状態で train/val を分ける
random.seed(tr_cfg.seed)
random.shuffle(metadata)
valid_meta = metadata[:tr_cfg.val_speakers]
#train_meta = metadata[:tr_cfg.val_speakers] # データセットの一部分で最初にコードを検証するときに使う
train_meta = metadata[tr_cfg.val_speakers:]

trainset = VCDataset(
    train_meta, #  全ての「話者＆発話」を一覧できる dict 形式データ
    n_utterances = tr_cfg.n_utterances, # 1 回の呼び出しでサンプルしたい 1 話者分の発話数（データセットに実際含まれる数ではない）
    sampling_rate = tr_cfg.sr, # ネットワークに流すための事前整形で目標とするサンプリング周波数
    min_sec = tr_cfg.min_sec, # 教師データの wav が最低限満たすべき有効な秒数
    sec = tr_cfg.sec, # サンプル秒数を一意に定める場合はここに秒数（例： 2.0）を指定
    max_sec = tr_cfg.max_sec, # sec = None の場合に最長限度とするサンプル秒数
    with_ref = True, # 同じ話者からランダムに選んだ別の発話も reference として持ってくる。バッチサイズは同じ。
    valid_mode = False, # モードが train か valid か。
)

validset = VCDataset(
    valid_meta,
    n_utterances = tr_cfg.n_utterances, 
    sampling_rate = tr_cfg.sr,
    min_sec = tr_cfg.min_sec, 
    sec = None, # None なら max_sec に合わせる
    max_sec = tr_cfg.max_sec, # sec = None の場合に最長限度とするサンプル秒数
    valid_mode = True, # モードが train か valid か。
)

assert len(trainset) >= tr_cfg.batch_size
assert len(validset) >= tr_cfg.batch_size
logger.info('Use {} speakers for training.'.format(len(trainset)))
logger.info('Use {} speakers for validation.'.format(len(validset)))

collater = WavCollator()

# DataLoader は train と val がそれぞれ必要
train_loader = DataLoader(
    trainset, 
    num_workers = tr_cfg.n_workers, 
    shuffle = True,
    sampler = None, # サンプリングではなく冒頭から振り出していく
    batch_size = tr_cfg.batch_size,
    pin_memory = False, # デフォルト False
    drop_last = True, # batch size で区切っていったときの端数を訓練に使わない
    collate_fn = collater,
)

valid_loader = DataLoader(
    validset, 
    num_workers = tr_cfg.n_workers, 
    shuffle = False, # データセット内での話者順をランダム化せずに冒頭から振り出す
    sampler = None,
    batch_size = tr_cfg.val_batch_size,
    pin_memory = False,
    drop_last = True,
    collate_fn = collater,
)

logger.info('Dataset and DataLoader are generated.')


In [ ]:
# データセットの最初のバッチを実際に振り出してみる

print("\n", "iter")
%time Iter = iter(train_loader)

print("\n", "next")
%time wavs, filenames, speakers, ref_wavs, ref_filenames = next(Iter)
print(wavs.shape) 
print(ref_wavs.shape)

from rich.pretty import pprint 
pprint(filenames)
pprint(ref_filenames)
pprint(speakers)



損失の定義。基本的に StyleTTS 2 から来ている


In [ ]:
# 損失関数、オプティマイザ、スケジューラのインスタンス化

from itertools import chain

import torch
from torch.optim import AdamW

from timm.scheduler.cosine_lr import CosineLRScheduler

from model.utils import scan_checkpoint
from model.StyleTTS2.losses import MultiResolutionSTFTLoss, GeneratorLoss, DiscriminatorLoss, WavLMLoss
from model.StyleTTS2.discriminators import MultiPeriodDiscriminator, MultiResSpecDiscriminator, WavLMDiscriminator


mpd = MultiPeriodDiscriminator(periods = [2, 3, 5, 7, 11, 17]).to(device)
msd = MultiResSpecDiscriminator()
wd = WavLMDiscriminator(slm_hidden = 768, slm_layers = 13, initial_channel = 64)

num_mpd_params = sum([p.numel() for p in list(mpd.parameters())])
logger.info(f"MultiPeriodDiscriminator has {num_mpd_params} parameters.")
num_msd_params = sum([p.numel() for p in list(msd.parameters())])
logger.info(f"MultiResSpecDiscriminator has {num_msd_params} parameters.")

# 48k 化する場合は fft の解像度を増やす必要がある。さもないと低音が評価から外れ、ピッチが変化しなくなるのでロボ声化する。
# 実は速度にはさほど影響しないので、もっと長い fft を掛けてもいいかも
stft_loss = MultiResolutionSTFTLoss(
    sample_rate = decoder_cfg.sampling_rate,
    fft_sizes = [2048, 1024, 512],
    win_lengths = [1200, 600, 240], # 24k 信号に対して [600, 1200, 240] = 25 ms, 50 ms, 10 ms
    hop_sizes = [240, 120, 50], # 24k 信号に対して [120, 240, 50] = 5 ms, 10 ms, 2.8 ms
).to(device)

gl = GeneratorLoss(mpd, msd).to(device)
dl = DiscriminatorLoss(mpd, msd).to(device)

# wavlm の重みをキャッシュにダウンロードするので 378 MB の通信が行われるっぽい
wl = WavLMLoss(
    'microsoft/wavlm-base-plus', 
    wd, 
    model_sr = tr_cfg.sr, # 24000 
    slm_sr = 16000 # SLM は入力信号の sr にかかわらず内部で 16k に自動変換されてから処理される
).to(device)

#### ここからエポックの定義

# 計算ステップと最終エポックはここで 0, -1 で初期化しておき、既存チェックポイントを読んで再開する場合は上書きする
step = 0
last_epoch = -1 # 最初は「前回 epoch = -1」なので、つまり今回 epoch = 0 の状態から開始
state_dict = None # 最初は前回のチェックポイントがないものとして扱う

# 学習再開のための state_dict 探索、ロード、ステップ数の調整

load_weight = False # 前回のチェックポイントを探索＆ロードするか。初回の場合は False に。

if load_weight:
    logging.info(f"Scanning checkpoints directory : {str(ckpt_path)}")
    LAST_CKPT_NAME = scan_checkpoint(str(ckpt_path), prefix = "vc_1st_") # ステップ数が大きい pt を自動探索
    if LAST_CKPT_NAME is not None:
        logger.info(f"Loading the last checkpoint {LAST_CKPT_NAME} ...")
        state_dict = torch.load(LAST_CKPT_NAME, map_location = device)
        style_encoder.load_state_dict(state_dict['style_encoder'])
        f0n_predictor.load_state_dict(state_dict['f0n_predictor'])
        decoder.load_state_dict(state_dict['decoder'])
        mpd.load_state_dict(state_dict['mpd'])
        msd.load_state_dict(state_dict['msd'])
        wd.load_state_dict(state_dict['wd'])
        step = state_dict['step'] + 1 # 前回最終状態に 1 足したステップから訓練開始。
        last_epoch = state_dict['epoch']
    else:
        logger.info(f"No checkpoints are found under {str(ckpt_path)}")


# オプティマイザをインスタンス化。GAN では g, do それぞれについて作成する（do は MPD と MSD の両方のパラメータを制御する）
# 学習可能なパラメータがない、もしくは重みを固定して使うネットワークは含めない
# 定義には、チューニング対象（通常は model オブジェクト）に存在するパラメータオブジェクトと、学習率を与える。

optimizer_g = AdamW(
    params = chain(
        style_encoder.parameters(), 
        f0n_predictor.parameters(), 
        decoder.parameters(),
    ), # 全層を対象にする場合
    lr = tr_cfg.max_lr, # 基本学習率
    betas = tr_cfg.betas,
)

optimizer_do = AdamW(
    params = chain(
        mpd.parameters(), 
        msd.parameters(), 
    ), # 全層を対象にする場合
    lr = tr_cfg.max_lr, # 基本学習率
    betas = tr_cfg.betas,
)

# オプティマイザの保存済みパラメータもロードする
if state_dict is not None:
    optimizer_g.load_state_dict(state_dict['optim_g'])
    optimizer_do.load_state_dict(state_dict['optim_do'])

####

# 続いてスケジューラを作成。継続の場合、オプティマイザの state_dict がロードされていないと正常に動かない。

scheduler_g = CosineLRScheduler(
    optimizer_g, 
    t_initial = tr_cfg.t_initial, 
    lr_min = tr_cfg.end_lr, 
    warmup_t = tr_cfg.n_warmup_epochs, 
    warmup_lr_init = tr_cfg.start_lr, 
    warmup_prefix = True, # Warmupが完了したタイミングの学習率を、オプティマイザーの基本学習率の設定値に合わせる
    cycle_decay = tr_cfg.cycle_decay, 
    k_decay = tr_cfg.k_decay,
    cycle_limit = tr_cfg.cycle_limit, # 1 だと SGDR の再起動を行わない
)

scheduler_do = CosineLRScheduler(
    optimizer_do, 
    t_initial = tr_cfg.t_initial, 
    lr_min = tr_cfg.end_lr, 
    warmup_t = tr_cfg.n_warmup_epochs, 
    warmup_lr_init = tr_cfg.start_lr, 
    warmup_prefix = True,
    cycle_decay = tr_cfg.cycle_decay, 
    k_decay = tr_cfg.k_decay,
    cycle_limit = tr_cfg.cycle_limit,
)

if state_dict is not None:
    scheduler_g.load_state_dict(state_dict['schedule_g'])
    scheduler_do.load_state_dict(state_dict['schedule_do'])
    logger.info(f"Resume training from epoch {last_epoch}, step {step}")
else:
    logger.info(f"Start training from epoch {last_epoch + 1}, step {step}")


In [ ]:
# スケジューラのプロット（model 用の utils で matplotlib.use("Agg") を使う場合はここより下で定義すること）

import matplotlib.pyplot as plt
lrs = []
for t in range(tr_cfg.n_epochs):
    lrs.append(scheduler_g._get_lr(t*1))

plt.plot(lrs)
plt.semilogy()
plt.show()


In [ ]:
# すでに走っていますと言われる場合、Reusing TensorBoard on port 6006 (pid 506885), started 4:34:13 ago. (Use '!kill 506885' to kill it.)
!rm -rf /tmp/.tensorboard-info/

%load_ext tensorboard
#reload_ext tensorboard
LOG_DIR = str(logs_path)
%tensorboard --logdir $LOG_DIR


In [ ]:
import os
from pathlib import Path
from fastprogress import master_bar, progress_bar
import time
from datetime import datetime

import torch
from torch.nn import functional as F
from torch.nn.utils import clip_grad_norm_
from torch.utils.tensorboard import SummaryWriter
import torchaudio

from model.utils import plot_spectrogram_harmof0

torch.multiprocessing.set_sharing_strategy('file_system')

resample_orig_to16 = torchaudio.transforms.Resample(orig_freq = tr_cfg.sr, new_freq = 16000).to(device)
resample_orig_to24 = torchaudio.transforms.Resample(orig_freq = tr_cfg.sr, new_freq = 24000)

writer = SummaryWriter(logs_path) # TensorBoard 用のイベントファイルを吐くオブジェクト

path_remote_log = os.path.join(".", "train_output.txt") # 損失値だけチラ見するためのログファイル

scaler = torch.cuda.amp.GradScaler() # 訓練開始時に一度、GradScaler をインスタンス化しておく

first_step = True # Ground truth を TensorBoard に書き出すために、最初の step のみ True とするフラグ

# 訓練時のプログレスバーを定義
mb = master_bar(range(max(0, last_epoch), tr_cfg.n_epochs)) # last_epoch は 最初は -1 なので、0 との max を取る必要がある。
total_start = time.time()

# epoch が「現在のエポック」を保持する変数。last_epoch は以下のイテレーション内では再代入されることはない。
for epoch in mb:
    # エポックの開始。epoch 変数は 0 始まりなので表示は + 1
    start = time.time()
    mb.main_bar.comment = ' Epoch: {:,d} / {:,d} (peak memory = {:5.2f} GB)'.\
            format(epoch + 1, tr_cfg.n_epochs, torch.cuda.max_memory_allocated()/1e9)
    pb = progress_bar(enumerate(train_loader), total = len(train_loader), parent = mb)

    #### Validation は valid_every エポックごとに実行する仕様に変更した
    if epoch % tr_cfg.valid_every == 0:
        style_encoder.eval() # 一時的に推論モードに
        f0n_predictor.eval()
        decoder.eval()
        val_loss_spec = 0.0
        val_loss_F0 = 0.0
        val_loss_norm = 0.0
        val_loss_content = 0.0
        val_loss_speaker = 0.0
        with torch.no_grad():
            for j, batch in enumerate(valid_loader):
                wav_orig = batch[0]
                filenames = batch[1] # ["path", "path", "path", "path"]
                speakers = batch[2] # ["name", "name", "name", "name"]
                # Valid dataset の順伝播。最初に 16k 化してHarmoF0 に通す
                w16 = resample_orig_to16(wav_orig.to(device))
                F0_real, act_t, N_real, spec = harmof0_tracker(w16)
                # pitch, energy の従来手法による推定値を得た後は、(acoustic) style encoder
                s_a = style_encoder(spec[:, 48:, :].unsqueeze(1)) # 4D にして投入
                # 次に ContentVec
                content = CE(w16)["last_hidden_state"].transpose(2, 1) # time last に転置
                # content が計算できたら、pitch と energy の予測値を作る
                F0_fake, N_fake = f0n_predictor(
                    content, # (batch, d_model = 768, n_frame) の time last
                    s_a, # =  style embedding, torch.Size([1, 128])
                )
                reconst = decoder(
                    content, # (batch, 768, frame)
                    F0_fake[:, -content.size(2)*2:], # (batch, frame*2) 
                    N_fake[:, -content.size(2)*2:],  # (batch, frame*2)
                    s_a, # (batch, 128)
                ) # 再構成した音声は singleton dimension が入った (batch, 1, time*) 
                # なお再構成側が元音声よりも 1 フレーム分短くなる。これは無視する。

                # 損失の計算。Valid は spectrogram の再構成損失だけを計算
                val_loss_spec += stft_loss(reconst.squeeze(), wav_orig[:, -reconst.shape[-1]:].detach().to(device))
                avg_val_loss = val_loss_spec / (j + 1) # val エラー率の平均値
                # 次に F0 と N の損失だが、実は real の方が 3 フレーム長い
                # リアルタイム VC だと基本的に末尾（時間的に新しい）を採用するので、こちらもそれに合わせる。
                val_loss_F0 +=  F.smooth_l1_loss(F0_real[:, -content.size(2)*2:], F0_fake[:, -content.size(2)*2:]) # F0 予測値の差
                avg_val_F0 = val_loss_F0 / (j + 1)
                val_loss_norm += F.smooth_l1_loss(N_real[:, -content.size(2)*2:], N_fake[:, -content.size(2)*2:]) # Energy 予測値の差
                avg_val_norm = val_loss_norm / (j + 1)
                mb.child.comment = f'Validating (epoch = {epoch:,d}, val loss = {avg_val_loss:4.3f})'
                
                # （valid_loader が吐き出す最初のバッチのみ）val の実際の出力データを SummaryWriter に流す
                if j == 0:
                    for k in range(min(10, tr_cfg.val_batch_size)):
                        spkr = speakers[k]
                        fpath = filenames[k]
                        fname = Path(fpath).name
                        if first_step == True:
                            writer.add_audio(
                                f'spkr-{spkr}_{fname}/x_(ground_truth)', 
                                wav_orig[k, :].detach().cpu(), 
                                epoch, 
                                tr_cfg.sr,
                            )
                            writer.add_figure(
                                f'spkr-{spkr}_{fname}/y_(ground_truth)', 
                                plot_spectrogram_harmof0(
                                    spec[k, 48:, :].detach().cpu().numpy(),
                                    f0 = F0_real[k, :].detach().cpu(), 
                                    act = act_t[k, :].detach().cpu(), 
                                ), 
                                epoch,
                            )
                            
                        writer.add_audio(
                            f'spkr-{spkr}_{fname}/xhat_(VC)', 
                            reconst[k, 0, :].detach(), 
                            epoch, 
                            tr_cfg.sr,
                        )
                        # 復元した wav (24k) から再度スペクトログラムを計算
                        w16_recon = resample_orig_to16(reconst.to(device))
                        F0_recon, _, _, spec_recon = harmof0_tracker(w16_recon) 
                        writer.add_figure(
                            f'spkr-{spkr}_{fname}/yhat_(VC)',
                            plot_spectrogram_harmof0(
                                spec_recon[k, 0, 48:, :].detach().cpu().numpy(),
                                f0 = F0_fake[k, :].detach().cpu(), 
                            ), 
                            epoch,
                        )
                first_step = False # 初回 validation step の処理が終わった
        style_encoder.train() # 保存後、訓練モードに戻す
        f0n_predictor.train()
        decoder.train()

        with open(path_remote_log, mode = 'a') as f:
            f.write(
                "[{}] Val spectrogram loss: {:5.4f} (epoch {:4d}, step {:7d})\n".format(
                    str(datetime.now()), avg_val_loss, epoch, step
                ),
            )
        # TensorBoard への valid 結果の書き込み
        if avg_val_loss > 0:
            writer.add_scalars(
                'Loss/00_spec_loss', {
                    "val": avg_val_loss.item()
                }, step
            )
            writer.add_scalars(
                'Loss/10_F0', {
                    "val": avg_val_F0.item()
                }, step
            )
            writer.add_scalars(
                'Loss/20_Energy', {
                    "val": avg_val_norm.item()
                }, step
            )
        writer.add_scalar("Monitor/max_memory_allocated", torch.cuda.max_memory_allocated() / 1e9, step)
        logging.info(f"Validation spec loss at epoch {epoch:10d}: {avg_val_loss:5.4f}")

    # tr_cfg.save_every ごとに、再学習が可能なフルセットの state_dict を保存する
    
    if epoch % tr_cfg.save_every == 0 and epoch != last_epoch:
        save_path = ckpt_path / f"vc_1st_{epoch:08d}.pt"
        style_encoder.cpu()
        f0n_predictor.cpu()
        decoder.cpu()
        torch.save(
            {
                'style_encoder': style_encoder.state_dict(),
                'f0n_predictor': f0n_predictor.state_dict(),
                'decoder': decoder.state_dict(),
                'optim_g': optimizer_g.state_dict(),
                'optim_do': optimizer_do.state_dict(),
                'mpd': mpd.state_dict(),
                'msd': msd.state_dict(),
                'wd': wd.state_dict(),
                'schedule_g': scheduler_g.state_dict(),
                'schedule_do': scheduler_do.state_dict(),
                'step': step,
                'epoch': epoch, 
            }, 
            save_path,
        )
        style_encoder.to(device)
        f0n_predictor.to(device)
        decoder.to(device)

    #### ここから各ステップの順伝播
    
    # DataLoader をラップしたプログレスバーで、訓練データセットから batch_size 話者、n_utterances 個の spec データを振り出す
    for i, batch in pb:
        start_b = time.time()
        wav_orig = batch[0]
        filenames = batch[1] # ["path", "path", "path", "path"]
        speakers = batch[2] # ["name", "name", "name", "name"]
        # 参照用に、同じ話者の別の発話も取り出す。追加のスタイル損失の計算に使う
        ref_wav_orig = batch[3]
        ref_filenames = batch[4] # ["path", "path", "path", "path"]

        #### 順伝播と損失の計算
        
        w16 = resample_orig_to16(wav_orig.to(device))
        with torch.no_grad():
            F0_real, act_t, N_real, spec = harmof0_tracker(w16)
        s_a = style_encoder(spec[:, 48:, :].unsqueeze(1))
        with torch.no_grad():
            content = CE(w16)["last_hidden_state"].transpose(2, 1)
                
        # スタイル損失の計算用に、別の発話の s_a だけ計算しておく。
        with torch.no_grad():
            ref_w16 = resample_orig_to16(ref_wav_orig.to(device))
            _, _, _, ref_spec = harmof0_tracker(ref_w16)
            ref_s_a = style_encoder(ref_spec[:, 48:, :].unsqueeze(1)) 

        F0_fake, N_fake = f0n_predictor(
            content, 
            s_a,
        )

        # decoder に長い音声を突っ込むと VRAM が足りないので、末尾からのランダムな開始場所で切り出した区間を使う
        decode_len = content.size(2) // 2 # 切り出す長さは元クリップの半分
        sample_from = random.randint(1, decode_len) # もし 0 が入ると decoder のスライスが狂うので 1 以上
        # 正しい F0 ベースの復元音声もしくは予想した F0 ベースの復元音声を作る。いずれも s_a をスタイルに使う
        if epoch >= tr_cfg.TMA_epoch and random.randint(0, 1) >= 1:
            reconst = decoder(
                content[:, :, -(sample_from+decode_len):-sample_from], 
                F0_fake[:, -2*(sample_from+decode_len):-2*sample_from], 
                N_fake[:, -2*(sample_from+decode_len):-2*sample_from], 
                s_a,
            ) # 再構成した音声は singleton dimension が入った (batch, 1, time*) 
        else:
            reconst = decoder(
                content[:, :, -(sample_from+decode_len):-sample_from], 
                F0_real[:, -2*(sample_from+decode_len):-2*sample_from], 
                N_real[:, -2*(sample_from+decode_len):-2*sample_from], 
                s_a,
            )

        # wav_orig は損失計算までに device に送っておく
        wav_orig = wav_orig[:, -decoder_cfg.upsample_total*(sample_from+decode_len):-decoder_cfg.upsample_total*sample_from].to(device) 
        
        ####

        # discriminator loss
        if epoch >= tr_cfg.TMA_epoch:
            optimizer_do.zero_grad()
            d_loss = dl(wav_orig.detach().unsqueeze(1).float(), reconst.detach()).mean() # こちらは fake ベース
            # 逆伝播→ unscale_() → clip_grad_norm_() → step() → update() の順
            scaler.scale(d_loss).backward()
            scaler.unscale_(optimizer_do)
            _ = clip_grad_norm_( mpd.parameters(), max_norm = tr_cfg.grad_clip, norm_type = 2.0, )
            _ = clip_grad_norm_( msd.parameters(), max_norm = tr_cfg.grad_clip, norm_type = 2.0, )
            _ = clip_grad_norm_( wd.parameters(), max_norm = tr_cfg.grad_clip, norm_type = 2.0, )
            scaler.step(optimizer_do) # scaler.step() は勾配の Inf や NaN をチェックし、問題なければ optimizer.step() を呼ぶ
            scaler.update() # 次のイテレーションに入るまでに、scaler をアップデート
        else:
            d_loss = 0

        # generator loss
        optimizer_g.zero_grad()
        loss_spec = stft_loss(reconst.squeeze(1), wav_orig[:, -reconst.shape[-1]:].detach())
        # F0 loss ここを、「activation map の値が大きいときだけ損失定義」として改造した
        act_binary = (act_t[:, -content.size(2)*2:] >= tr_cfg.f0_act_threshold).float()
        loss_F0_rec =  F.smooth_l1_loss(
            act_binary * F0_real.detach()[:, -content.size(2)*2:], 
            act_binary * F0_fake[:, -content.size(2)*2:]
        )
        loss_norm_rec = F.smooth_l1_loss(N_real[:, -content.size(2)*2:].detach(), N_fake[:, -content.size(2)*2:]) # Energy 予測値の差        

        loss_sty = F.l1_loss(s_a.detach(), ref_s_a)
        
        if epoch >= tr_cfg.TMA_epoch: # エポックが TMA_epoch 以上にならないと TMA が開始されない
            loss_gen_all = gl(wav_orig.detach().unsqueeze(1).float(), reconst).mean() # stft loss は第 1 引数が reconst で gl, wl は逆
            loss_slm = wl(wav_orig.detach(), reconst.squeeze(1)).mean()
            g_loss = tr_cfg.lambda_spec * loss_spec + \
                tr_cfg.lambda_F0 * loss_F0_rec + \
                tr_cfg.lambda_norm * loss_norm_rec + \
                tr_cfg.lambda_sty * loss_sty + \
                tr_cfg.lambda_gen * loss_gen_all + \
                tr_cfg.lambda_slm * loss_slm
        else:
            loss_gen_all = 0
            loss_slm = 0
            g_loss = tr_cfg.lambda_spec * loss_spec + \
                tr_cfg.lambda_F0 * loss_F0_rec + \
                tr_cfg.lambda_norm * loss_norm_rec + \
                tr_cfg.lambda_sty * loss_sty
        
        scaler.scale(g_loss).backward()
        scaler.unscale_(optimizer_g)
        _ = clip_grad_norm_( style_encoder.parameters(), max_norm = tr_cfg.grad_clip, norm_type = 2.0, )
        _ = clip_grad_norm_( decoder.parameters(), max_norm = tr_cfg.grad_clip, norm_type = 2.0, )
        scaler.step(optimizer_g)
        scaler.update()
        
        # 子プログレスバーのコメントを更新
        mb.child.comment = '(step {:09d}, spectrogram loss: {:03.3f}, {:03.3f} steps/s)'. \
                format(step, loss_spec.item(), 1 / (time.time() - start_b) )

        if step % tr_cfg.board_every == 0:
            with torch.no_grad():
                writer.add_scalars(
                    'Loss/00_spec_loss', {
                        "train": loss_spec.item()
                    }, step
                )
                writer.add_scalars(
                    'Loss/10_F0', {
                        "train": loss_F0_rec.item()
                    }, step
                )
                writer.add_scalars(
                    'Loss/20_Energy', {
                        "train": loss_norm_rec.item()
                    }, step
                )
                if epoch >= tr_cfg.TMA_epoch: 
                    writer.add_scalar("Loss/30_train_gen_all", loss_gen_all.item(), step)
                    writer.add_scalar("Loss/40_train_slm", loss_slm.item(), step)
                writer.add_scalar("Loss/50_train_sty", loss_sty.item(), step)
                writer.add_scalar("Monitor/lr", optimizer_g.param_groups[0]["lr"], step)
                writer.add_scalar("Monitor/max_memory_allocated", torch.cuda.max_memory_allocated() / 1e9, step)
        torch.cuda.empty_cache()
        step += 1 # オプティマイザとは関係なくユーザーが作った、現在のステップを保持する変数をインクリメント

    # 以下は各エポックの全 step 投入後に行う処理
    scheduler_g.step(epoch + 1) # 学習率の更新。更新単位は通常は epoch
    scheduler_do.step(epoch + 1)

# 当初予定のエポックが全て終了したら値を保存する
save_path = ckpt_path / f"vc_1st_{epoch:08d}.pt"
style_encoder.eval().cpu()
f0n_predictor.eval().cpu()
decoder.eval().cpu()
msd.eval()
mpd.eval()
torch.save(
    {
        'style_encoder': style_encoder.state_dict(),
        'f0n_predictor': f0n_predictor.state_dict(),
        'decoder': decoder.state_dict(),
        'optim_g': optimizer_g.state_dict(),
        'optim_do': optimizer_do.state_dict(),
        'mpd': mpd.state_dict(),
        'msd': msd.state_dict(),
        'wd': wd.state_dict(),
        'schedule_g': scheduler_g.state_dict(),
        'schedule_do': scheduler_do.state_dict(),
        'step': step,
        'epoch': epoch, 
    }, 
    save_path,
)

logging.info(f"Epoch {epoch:10d} ended.")
